In [18]:
# interactive plot to check DVN results
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from plotly.subplots import make_subplots
import pandas as pd
import time
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, SpectralEmbedding
import plotly.express as px
import plotly.graph_objs as go
import ipywidgets as widgets
import os
from sklearn import preprocessing
from ast import literal_eval
import io
from PIL import Image
import librosa
import sys
sys.path.append("..")
import torch
from feature_extraction import pytorch_utils as pu
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from IPython.display import Audio
import os, random
from torch.utils.data import Dataset, DataLoader

min_max_scaler = preprocessing.MinMaxScaler()
n_neighbors = 10
FREQ_BINS=30
TIME_STEPS=20
SR=44100

def plotly_able_df(df,num_d=2):
    
    df["feats"]=df["feats"].apply(literal_eval)
    X=df["feats"]
    #convert series of arrays into a numpy array
    X=pd.DataFrame(X.to_list()).to_numpy()
    X = min_max_scaler.fit_transform(X)
    time_start = time.time()
#     tsne = TSNE(n_components=num_d, random_state=0, perplexity=100, verbose=1)
    tsne = Isomap(n_neighbors, n_components=num_d)
#     tsne = LocallyLinearEmbedding(n_neighbors,n_components=num_d)
#     tsne = SpectralEmbedding(n_neighbors=10,n_components?=num_d,random_state=1)
    X_2d = tsne.fit_transform(X)
    
    if num_d==2:
        df2=pd.concat([df["path"],df["label"],df["drum_type"],pd.Series(X_2d[:,0]),pd.Series(X_2d[:,1])],axis=1)
        df2.columns=["path","label","drum_type","D1","D2"]
    if num_d==3:
        df2=pd.concat([df["path"],df["label"],df["drum_type"],pd.Series(X_2d[:,0]),pd.Series(X_2d[:,1]),pd.Series(X_2d[:,2])],axis=1)
        df2.columns=["path","label","drum_type","D1","D2","D3"]
    
    df2.label = df2.label.astype('str')
    return df2

class interactive_graph():
    def hover_fn(self,trace, points, state):
        if points.point_inds:
            ind = points.point_inds[0]
            drmName=trace.customdata[ind][0][:]
            filename=os.getcwd()+"/"+drmName
            with open(filename,'rb') as f:
                audio_data = f.read()
            self.aud.value=audio_data
            self.hover_data.value = str(drmName)+"\n"
            self.audioImg.value=self.audDisplay(filename)
            
    def audDisplay(self,f):
        #this got annoying because widget only accepts byte version of images
        audio_array=librosa.load(f)
        signals=audio_array[0]
        nz=np.max((SR-signals.shape[0],0))
        signals=np.concatenate([signals[0:SR],np.zeros(nz)]).astype("float32")

        sound={"signal":torch.tensor(signals),"label":'',"path":'',"drum_type":''}
        trns=pu.specTrans(FREQ_BINS,time_steps=TIME_STEPS)
        ft=trns(sound)["feats"]
        sf=ft.detach().numpy()[0]
        #flip upside down
        sf=sf[-1:0:-1][:]
        x=plt.imshow(sf)
        #convert to bytes so can be set to widget data
        buf = io.BytesIO()
        x.figure.savefig(buf, format='png')
        buf.seek(0)
        bufD=buf.getvalue()
        buf.close()
        
        return bufD
    def __init__(self,df,grouping_by="label",title="",num_d=2,sym_size=10,color=px.colors.qualitative.Vivid):
        if num_d==2:
            p = px.scatter(df, x="D1",y="D2",symbol="drum_type",color=grouping_by,hover_data=["path"],color_discrete_sequence=color)
        elif num_d==3:
            p = px.scatter_3d(df, x="D1",y="D2",z="D3",color=grouping_by,hover_data=["path"],symbol="label",color_discrete_sequence=color)
        for trace in p.data:
            trace.update(hoverinfo="none",hovertemplate= '')
        p.update_traces(marker=dict(size=sym_size,
                              line=dict(width=0,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

        self.hover_data = widgets.Textarea()  
        #audio and img widgets
        self.aud=widgets.Audio(autoplay=True,loop=False,embedding=True)
        self.audioImg=widgets.Image(
            value=b'',
            format='png',
            width='30%', 
        )
        #####
        layout = go.Layout(hovermode=False,)
        self.fig  = go.FigureWidget(p)
        self.fig.update_layout(scene = dict(
                    camera=dict(eye=dict(x=-1, y=-1, z=0)),
                    aspectmode="cube",),
                    margin=dict( r=0, l=0, b=0, t=0))
                    
        for f in self.fig.data:
            f.on_hover(self.hover_fn)        


df = pd.read_csv("feature_extraction/csvs/best64_60.csv")
df["path"] = "feature_extraction/drum_dbs/"+df["path"]

# df_64_3d=plotly_able_df(pd.read_csv("feature_extraction/csvs/spec_encode_64_290_0.002042.pt.csv"),num_d=3)
# df_8_3d=plotly_able_df(pd.read_csv("feature_extraction/csvs/spec_encode_sm_8_270_0.003504.pt.csv"),num_d=3,)
# df_64_2d=plotly_able_df(pd.read_csv("feature_extraction/csvs/spec_encode_64_290_0.002042.pt.csv"),num_d=2)
df_8_2d=plotly_able_df(df,num_d=2,)


In [19]:
df_8_2d


,path,label,drum_type,D1,D2
0,feature_extraction/drum_dbs/./dk_data/tom_high...,0,tom_high,-0.244804,-0.077796
1,feature_extraction/drum_dbs/./dk_data/tom_high...,0,tom_high,-0.233414,-0.189368
2,feature_extraction/drum_dbs/./dk_data/tom_high...,0,tom_high,0.387462,0.013692
3,feature_extraction/drum_dbs/./dk_data/tom_high...,0,tom_high,-0.195168,-0.043961
4,feature_extraction/drum_dbs/./dk_data/tom_high...,0,tom_high,-0.304321,-0.120542
...,...,...,...,...,...
4282,feature_extraction/drum_dbs/./dk_data/synth_no...,1,synth_noise,-0.314653,0.024973
4283,feature_extraction/drum_dbs/./dk_data/synth_no...,1,synth_noise,0.676265,0.042728
4284,feature_extraction/drum_dbs/./dk_data/synth_no...,1,synth_noise,0.192355,-0.095037
4285,feature_extraction/drum_dbs/./dk_data/synth_no...,1,synth_noise,-0.202883,-0.024380


In [20]:
# df_8_3d.to_csv("./feature_extraction/csvs/df_8_3D.csv",index=False)
# df_64_3d.to_csv("./feature_extraction/csvs/df_64_3D.csv",index=False)
# df_8_2d.to_csv("./feature_extraction/csvs/df_8_2D.csv",index=False)
# df_64_2d.to_csv("./feature_extraction/csvs/df_64_2D.csv",index=False)
# df_8_2d

In [21]:
ig=interactive_graph(df_8_2d,grouping_by="drum_type",title="8 dim embedding",num_d=2,sym_size=5,color=px.colors.qualitative.Dark24_r,)

display(ig.fig,widgets.HBox([ig.hover_data,ig.aud,ig.audioImg]))

FigureWidget({
    'data': [{'customdata': array([['feature_extraction/drum_dbs/./dk_data/tom_high/Roland Tr-9…